In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import linear_model
import random
from sklearn.metrics import r2_score
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Financial\ Modeling/CMF/Хакатон\ от\ Сбера
!pwd

In [ ]:
orders = pd.read_csv('orders.csv')
daily_orders = pd.read_csv('orders.csv')
daily_orders['date'] = pd.to_datetime(daily_orders['date'])
orders['date'] = pd.to_datetime(orders['date'])
partners_delays = pd.read_csv('partners_delays.csv')
partners_delays['dttm'] = pd.to_datetime(partners_delays['dttm'])
clusters = pd.read_excel('standard_scaler_clustering.xlsx')
partners_delays

In [ ]:
delivery_area_id = 34

df = orders[(orders['delivery_area_id'] == delivery_area_id) &
            (orders['date'].dt.date == pd.to_datetime('2021-10-20'))].copy()
df

In [ ]:
orders['time'] = pd.to_datetime(orders['date']).dt.hour
orders

In [ ]:
orders.columns = ['delivery_area_id', 'date_time', 'orders_cnt', 'time']
orders

In [ ]:
orders['date'] = pd.to_datetime(orders['date_time']).dt.date
orders

In [ ]:
def norm(df):
  """
  """
  df_ = df.copy()
  sum = df['orders_cnt'].sum()
  df_['orders_cnt'] = df_['orders_cnt'] / sum
  df_['norm'] = sum

  return df_

test = orders.groupby(['delivery_area_id', 'date']).apply(norm)
test

In [ ]:
plt.hist(test['norm'], bins=70);

In [ ]:
plt.scatter(test['time'], test['orders_cnt']);

In [ ]:
plt.figure(figsize=(24,10))
for i in range(test['time'].value_counts().shape[0]):
  h = test['time'].value_counts().index[i]
  ax = plt.subplot(4, 5, i+1)
  ax.set_title(h)
  ax.hist(test['orders_cnt'].loc[test['time'] == h], bins=30)

In [ ]:
mean = test.groupby('time')['orders_cnt'].mean()
mean

In [ ]:
plt.plot(mean)

In [ ]:
plt.yscale('log')
plt.hist(partners_delays['delay_rate']);

In [ ]:
partners_delays[['delivery_area_id','dttm']].loc[
    partners_delays['delay_rate'] > .8].value_counts().index

In [ ]:
plt.yscale('log')
plt.hist(partners_delays['delay_rate'], bins=20)

In [ ]:
partners_delays.loc[partners_delays['delay_rate'] >= .05]

In [ ]:
no_delay = partners_delays.loc[(partners_delays['delay_rate'] < .05) &
                               (partners_delays['dttm'].isin(orders['date_time']))]
orders_
no_delay                            

In [ ]:
orders

In [ ]:
delivery_area_id = 206
orders_test = orders.loc[(orders['delivery_area_id'] == delivery_area_id)]
delay_rate_test = partners_delays.loc[(partners_delays['delivery_area_id'] == delivery_area_id)]
delay_rate_test = delay_rate_test[delay_rate_test['dttm'].isin(orders_test['date_time'])]
delay_rate_test['orders_cnt'] = orders_test['orders_cnt'].values

orders_test.shape, delay_rate_test.shape

In [ ]:
delay_rate_test

In [ ]:
plt.scatter(delay_rate_test['orders_cnt'], delay_rate_test['partners_cnt'])
plt.scatter(delay_rate_test['orders_cnt'].loc[delay_rate_test['delay_rate'] > 0.05],
            delay_rate_test['partners_cnt'].loc[delay_rate_test['delay_rate'] > 0.05])

In [ ]:
delay_rate_test['orders_cnt'].loc[delay_rate_test['delay_rate'] > 0.05]

In [ ]:
delay_rate_test['Target'] = (delay_rate_test['delay_rate'] > 0.05).astype(int)
delay_rate_test

In [ ]:
from fastai.tabular.core import add_datepart

add_datepart(delay_rate_test, 'dttm', drop=False, time=True)
delay_rate_test

In [ ]:
delay_rate_test.columns
delay_rate_test.drop(['delivery_area_id', 'dttmYear', 'dttmDayofyear'], axis=1,
                     inplace=True)
delay_rate_test

In [ ]:
from sklearn.model_selection import train_test_split

X = delay_rate_test.drop(['dttm', 'dttmMonth', 'delay_rate',
                          'dttmMinute', 'dttmSecond', 'dttmElapsed', 'Target',
                          'dttmMinute',	'dttmSecond', 'dttmIs_year_start'],
                         axis=1)
y = delay_rate_test['Target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                                  random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
X.columns

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

model = RandomForestClassifier(n_estimators = 100,
                               max_depth = 5,
                               max_features = int(np.sqrt(X_train.shape[1]))
                                    )
model.fit(X_train, y_train)
roc_auc_score(y_val, model.predict_proba(X_val)[:,1])

In [ ]:
model.predict_proba(X_val)

In [ ]:
roc_auc_score(y_test, model.predict(X_test))

In [ ]:
feature_importances = (model.feature_importances_ / sum(model.feature_importances_)) * 100

results = pd.DataFrame({'Features': X_train.columns,
                        'Importances': feature_importances})
results.sort_values(by='Importances', inplace=True)
large = 20; med = 16; small = 12;
params = {'axes.titlesize': large,
          'figure.figsize': (13, 7),
          'axes.labelsize': large,
          'axes.titlesize': large,
          'xtick.labelsize': large,
          'ytick.labelsize': large,
          'figure.titlesize': large}
plt.rcParams.update(params)

plt.figure(figsize=(15,10))
ax = plt.barh(results['Features'], results['Importances'])
plt.xlabel('Importance percentages')
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

model = DecisionTreeClassifier(max_depth = 4
                                    )
model.fit(X_train, y_train)
print(roc_auc_score(y_val, model.predict(X_val)))
print(roc_auc_score(y_test, model.predict(X_test)))